In [ ]:
import numpy as np
import xarray as xr
import os,errno
import sys
import datetime as dt
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd
%matplotlib inline
dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/ghcnd_all/'#edited
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'

In [ ]:
if 1==1:#step 1: get stations with 95% completeness from 1979 - 2019 and data from Sep 1979 and Nov 2019
    ds=pd.read_csv(dir1+'ghcnd-stations_new.txt',header=None)
    print(ds)
    count=0
    yr_start=1979
    yr_end=2019
    count=0
    count_ne=0
    stations=[]
    lats=[]
    lons=[]
    elevs=[]
    station_names=[]
    thresholds_annual_99=[]
    thresholds_annual_95=[]
    thresholds_winter_99=[]
    thresholds_spring_99=[]
    thresholds_summer_99=[]
    thresholds_fall_99=[]
    thresholds_winter_95=[]
    thresholds_spring_95=[]
    thresholds_summer_95=[]
    thresholds_fall_95=[]
    for row in ds.iterrows():
        if count%500==0:
            print(count)
        count=count+1
        if row[1][0][0:2]=='US':
            filename=row[1][0][0:11]
            rest_space=row[1][0][11:]
            split=rest_space.split()
            
            lat= float(split[0])
            lon= float(split[1])
            elev=float(split[2])
            state=split[3]
            station_name=split[4]
            
            if station_name in ['MT','NEW'] or station_name[0]==station_name:#just trying to get more unique station names but NOT a complete set; use filenames which are unique
                try:
                    station_name=station_name+' '+split[5]
                except:
                    pass

            station_points=[(-73.8092, 42.7431),(-73.8842, 41.6267),(-75.985, 42.1997),(-74.155, 41.7681),(-74.6142, 41.5917),(-73.9692, 40.7789),(-73.4167, 41.4),(-74.6625, 41.2214),(-75.4492, 40.6508),(-75.7269, 41.3336)]
            names=['']
            if state in ['MD','ME','NH','MA','RI','CT','VT','NY','NJ','PA','VA','WV','DE']:
                filename=dir1+row[1][0][0:11]+'.dly'
                years=[]
                months=[]
                obs_list=[]
                obs_winter=[]
                obs_spring=[]
                obs_summer=[]
                obs_fall=[]
                ds1=pd.read_csv(filename,header=None)#,sep=None)
                nancount=0
                qcount=0
                yescount=0
                for row1 in ds1.iterrows():
                    row_neg_space=row1[1][0].replace('-',' -')
                    row_basic=row1[1][0]
                    a=row_neg_space.split()
                    code=a[0][-4:]
                    if code=='PRCP':
                        
                        year=int(a[0][11:15])
                        month=int(a[0][15:17])
                        if yr_start<=year<=yr_end:
                            
                            years.append(year)
                            months.append(month)
                            if month in [1,3,5,7,8,10,12]:
                                days=31
                            if month in [4,6,9,11]:
                                days=30
                            if month==2 and year%4==0:
                                days=29
                            if month==2 and year%4!=0:
                                days=28
                            for i in range(days):
                                #print(row_basic)
                                mflag=row_basic[26+8*i]
                                qflag=row_basic[27+8*i]
                                sflag=row_basic[28+8*i]
                                obs=float(row_basic[21+8*i:26+8*i])
                                if obs==-9999.0:
                                    nancount=nancount+1
                                elif qflag!=' ':
                                    qcount=qcount+1
                                else:
                                    
                                    yescount=yescount+1
                                    obs_list.append(obs)
                                    
                                    if month in [12,1,2] and 1980<=year<=2018 or month==12 and year==1979 or month in [1,2] and year==2019:
                                        obs_winter.append(obs)
                                    if month in [3,4,5]:
                                        obs_spring.append(obs)
                                    if month in [6,7,8]:
                                        obs_summer.append(obs)
                                    if month in [9,10,11]:
                                        obs_fall.append(obs)
                                    #tuple=(year,month,i,obs)
                                    #print(tuple)
                            #month=int(a[0][-6:-4])
                
                ratio=float(yescount)/float((365*41+10))
                zip_years_months=list(zip(years,months))
                months_yr_start=[x[1] for x in zip_years_months if x[0]==yr_start]
                months_yr_end=[x[1] for x in zip_years_months if x[0]==yr_end]
                months_yr_start=list(set(months_yr_start))
                months_yr_end=list(set(months_yr_end))

            
                if len(years)>0 and ratio>=0.95 and len(months_yr_start)>0 and len(months_yr_end)>0:
                    if min(years)==yr_start and max(years)==yr_end and min(months_yr_start)<=9 and max(months_yr_end)==12:
                        
                        measurable=[x for x in obs_list if x>=1]#0.1mm = measurable
                        
                        measurable_winter=[x for x in obs_winter if x>=1]
                        measurable_spring=[x for x in obs_spring if x>=1]
                        measurable_summer=[x for x in obs_summer if x>=1]
                        measurable_fall=[x for x in obs_fall if x>=1]
                        
                        threshold_winter_99=np.percentile(measurable_winter,99,interpolation='linear')
                        threshold_spring_99=np.percentile(measurable_spring,99,interpolation='linear')
                        threshold_summer_99=np.percentile(measurable_summer,99,interpolation='linear')
                        threshold_fall_99=np.percentile(measurable_fall,99,interpolation='linear')
                        
                        threshold_winter_95=np.percentile(measurable_winter,95,interpolation='linear')
                        threshold_spring_95=np.percentile(measurable_spring,95,interpolation='linear')
                        threshold_summer_95=np.percentile(measurable_summer,95,interpolation='linear')
                        threshold_fall_95=np.percentile(measurable_fall,95,interpolation='linear')
                        
                        threshold_annual_99=np.percentile(measurable,99,interpolation='linear')
                        threshold_annual_95=np.percentile(measurable,95,interpolation='linear')
                        thresholds_annual_99.append(threshold_annual_99)
                        thresholds_annual_95.append(threshold_annual_95)
                        
                        thresholds_winter_99.append(threshold_winter_99)
                        thresholds_spring_99.append(threshold_spring_99)
                        thresholds_summer_99.append(threshold_summer_99)
                        thresholds_fall_99.append(threshold_fall_99)
                        
                        thresholds_winter_95.append(threshold_winter_95)
                        thresholds_spring_95.append(threshold_spring_95)
                        thresholds_summer_95.append(threshold_summer_95)
                        thresholds_fall_95.append(threshold_fall_95)
                        
                        stations.append(filename)
                        lats.append(lat)
                        lons.append(lon)
                        elevs.append(elev)
                        station_names.append(station_name)
                        count_ne=count_ne+1
    print(count_ne)          
    
    dk=xr.Dataset()
    
    dk['stations']=(('num'),stations)
    dk['lats']=(('num'),lats)
    dk['lons']=(('num'),lons)
    dk['elevs']=(('num'),elevs)
    dk['thresholds_winter_99']=(('num'),thresholds_winter_99)
    dk['thresholds_spring_99']=(('num'),thresholds_spring_99)
    dk['thresholds_summer_99']=(('num'),thresholds_summer_99)
    dk['thresholds_fall_99']=(('num'),thresholds_fall_99)
    
    dk['thresholds_winter_95']=(('num'),thresholds_winter_95)
    dk['thresholds_spring_95']=(('num'),thresholds_spring_95)
    dk['thresholds_summer_95']=(('num'),thresholds_summer_95)
    dk['thresholds_fall_95']=(('num'),thresholds_fall_95)
    
    dk['thresholds_annual_99']=(('num'),thresholds_annual_99)
    dk['thresholds_annual_95']=(('num'),thresholds_annual_95)
    dk['station_names']=(('num'),station_names)
    dk.coords['num']=np.arange(len(stations))
    print(station_names)
    print(dk['station_names'])
    try:
        os.remove(dir+'station_numbers_95.nc')#save eligible stations and corresponding thresholds, lats, lons in this file
    except OSError:
        pass
    dk.to_netcdf(dir+'station_numbers_95.nc',mode='w',format='NETCDF4',unlimited_dims='num')

In [ ]:
if 1==1:#part 2: read the obs at eligible stations into a better format
    yr_start=1979
    yr_end=2019
    tuples=[]
    years=[]
    months=[]
    days1=[]
    lats1=[]
    lons1=[]
    elevs1=[]
    observations=[]
    dates=[]
    station_names_list=[]
    stations_list=[]
    dp=xr.open_dataset(dir+'station_numbers_95.nc')
    stations=dp.stations.values.tolist()
    lats=dp.lats.values.tolist()
    lons=dp.lons.values.tolist()
    elevs=dp.elevs.values.tolist()
    station_names=dp.station_names.values.tolist()
    
    points=zip(lons,lats)
    for i in range(len(stations)):
        print(str(i+1)+'/'+str(len(stations)))
        lat=lats[i]
        lon=lons[i]
        elev=elevs[i]
        station_name=station_names[i]
        station=stations[i]
        filename=stations[i]
        ds1=pd.read_csv(filename,header=None)
        nancount=0
        qcount=0
        yescount=0
        for row1 in ds1.iterrows():
            row_neg_space=row1[1][0].replace('-',' -')
            row_basic=row1[1][0]
            a=row_neg_space.split()
            code=a[0][-4:]
            if code=='PRCP':
                year=int(a[0][11:15])
                if yr_start<=year<=yr_end:
                    month=int(a[0][15:17])
                    if month in [1,3,5,7,8,10,12]:
                        days=31
                    if month in [4,6,9,11]:
                        days=30
                    if month==2 and year%4==0:
                        days=29
                    if month==2 and year%4!=0:
                        days=28

                    for i in range(days):
                        #print(row_basic)
                        mflag=row_basic[26+8*i]
                        qflag=row_basic[27+8*i]
                        sflag=row_basic[28+8*i]
                        obs=float(row_basic[21+8*i:26+8*i])
                    
                        if obs==-9999.0:
                            pass
                        elif qflag!=' ':
                            pass
                        else:
                            #if 1980<=year<=2018 and month in [1,2,12] or year==1979 and month==12 or year==2019 and month in [1,2]:#winter
                            #if month in [9,10,11]:#fall
                            #if month in [6,7,8]:#summer
                            #if month in [3,4,5]:#spring
                            if 1==1:
                                years.append(year)
                                months.append(month)
                                days1.append(i)
                                observations.append(obs/10.)#convert from tenths of mm to mm
                                lats1.append(lat)
                                lons1.append(lon)
                                elevs1.append(elev)
                                date=dt.datetime(year,month,i+1,6)
                                dates.append(date)
                                station_names_list.append(station_name)
                                stations_list.append(station)

In [ ]:
print(thresholds_fall_99)#just checking threshold values
print(thresholds_annual_99)

In [ ]:
#for a range of station extreme and area sum thresholds: calculate extreme precip sums per day, then
#choose the top 5, 10, or 20% of extreme-precip-having days by the extreme precip sum.  These are 
#the "EP days".  

ds.close()
yrs_neusa=np.arange(1979,2020,1)
ds=xr.open_dataset(dir+'station_numbers_95.nc')
station_urls=ds['stations'] 
print(len(station_urls.values))

thresholds_fall_99=ds['thresholds_fall_99']/10.
thresholds_spring_99=ds['thresholds_spring_99']/10.
thresholds_summer_99=ds['thresholds_summer_99']/10.
thresholds_winter_99=ds['thresholds_winter_99']/10.
thresholds_fall_95=ds['thresholds_fall_95']/10.
thresholds_spring_95=ds['thresholds_spring_95']/10.
thresholds_summer_95=ds['thresholds_summer_95']/10.
thresholds_winter_95=ds['thresholds_winter_95']/10.
thresholds_annual_99=ds['thresholds_annual_99']/10.
thresholds_annual_95=ds['thresholds_annual_95']/10.

seasons_list=['winter','winter','spring','spring','summer','summer','fall','fall','annual','annual']

station_thresholds_list=[thresholds_winter_99,thresholds_winter_95,thresholds_spring_99,
                         thresholds_spring_95,thresholds_summer_99,thresholds_summer_95,
                         thresholds_fall_99,thresholds_fall_95,thresholds_annual_99,thresholds_annual_95]

print(len(station_thresholds_list))
area_thresholds_list=[80,90,95]
thresholds_list=[99,95,99,95,99,95,99,95,99,95]
zipped=list(zip(dates,lats1,lons1,observations,stations_list))#stations_list is urls too
threshold_dict={}
        
for l1 in range(len(station_thresholds_list)):
    
    thresholds_choose=station_thresholds_list[l1]
    for i in range(len(station_urls)):
        threshold_dict[station_urls.values[i]]=thresholds_choose[i]
    season_choose=seasons_list[l1]
    thresh_choose=thresholds_list[l1]

    if season_choose=='winter':
        for i in range(len(yrs_neusa)):
            year=yrs_neusa[i]
            if year==1979:
                pass
            if 1980<=year<=2019:
                starting_date=dt.datetime(year-1,12,1,6)
                if year%4==0:
                    date_range=[starting_date+dt.timedelta(x) for x in range(91)]
                else:
                    date_range=[starting_date+dt.timedelta(x) for x in range(90)]
            if i==1:
                all_dates=date_range
            else:
                all_dates=all_dates+date_range
    else:
        for i in range(len(yrs_neusa)):
            year=yrs_neusa[i]
            if season_choose=='fall':
                starting_date=dt.datetime(year,9,1,6)
                date_range=[starting_date+dt.timedelta(x) for x in range(91)]
            if season_choose=='spring':
                starting_date=dt.datetime(year,3,1,6)
                date_range=[starting_date+dt.timedelta(x) for x in range(92)]
            if season_choose=='summer':
                starting_date=dt.datetime(year,6,1,6)
                date_range=[starting_date+dt.timedelta(x) for x in range(92)]
            if season_choose=='annual':
                starting_date=dt.datetime(year,9,1,6)
                date_range=[starting_date+dt.timedelta(x) for x in range(91)]
            if i==0:
                all_dates=date_range
            else:
                all_dates=all_dates+date_range
    #print(all_dates)
    daily_extreme_totals=[]
    for i in range(len(all_dates)):#now find extreme sums for each day
        if i%100==0:
            print(i)
        select_day=[x for x in zipped if x[0]==all_dates[i]]
        daily_extreme_sum=0
        count_day_values=0#how many valid observations are there?
        for j in range(len(select_day)):
            station_url=select_day[j][-1]
            obs=select_day[j][-2]

            if obs>=0:
                count_day_values=count_day_values+1
            threshold=threshold_dict[station_url]
            if obs>=threshold:#add extreme values
                daily_extreme_sum=daily_extreme_sum+obs
            if j==len(select_day)-1:
                if count_day_values/float(len(station_urls))>=0.90:#only count days with 90%+ data coverage (should be almost all days)
                    daily_extreme_totals.append(float(daily_extreme_sum)/float(count_day_values))
                else:
                    daily_extreme_totals.append(np.nan)
    
    for l2 in range(len(area_thresholds_list)):
        
        area_threshold_choose=area_thresholds_list[l2]
        
        print(thresholds_choose)
        print(area_threshold_choose)
        print(season_choose)
        print(thresh_choose)
        
        print('MAX THRESH')
        print(max(thresholds_choose).values)
        print(len(all_dates))
        print(len(daily_extreme_totals))
        print(len([x for x in daily_extreme_totals if x>0]))
                        
        zipped_sums=list(zip(all_dates,daily_extreme_totals))
        nonzero_extremes=[x for x in daily_extreme_totals if x>0]
        extreme_sum_threshold=np.percentile(nonzero_extremes,area_threshold_choose,interpolation='linear')
        zipped_select=[x for x in zipped_sums if x[1]>=extreme_sum_threshold]
        dates_extreme=[x[0] for x in zipped_select]
        print(len(dates_extreme))
        data_select=[x for x in zipped if x[0] in dates_extreme]
        dates_select=[x[0] for x in data_select]
        lats_select=[x[1] for x in data_select]
        lons_select=[x[2] for x in data_select]
        obs_select=[x[3] for x in data_select]  
        stations_select=[x[4] for x in data_select]
        
        dw=xr.Dataset()
        dw['lats']=(('station'),lats_select)
        dw['lons']=(('station'),lons_select)
        dw['obs']=(('station'),obs_select)
        dw['dates']=(('station'),dates_select)
        dw['stations']=(('station'),stations_select)
        dw.coords['station']=np.arange(len(lats_select))
        
        #save set of extreme days with corresponding filename
        try:
            os.remove(dir+'extreme_days_ghcnd_'+str(thresh_choose)+'_'+str(area_threshold_choose)+'_'+season_choose+'.nc')
            #os.remove(dir+'extreme_days_ghcnd_fall_99_80.nc')
        except OSError:
            pass
        #dw.to_netcdf(dir+'extreme_days_ghcnd_fall_99_80.nc',mode='w',format='NETCDF4',unlimited_dims='station')
        dw.to_netcdf(dir+'extreme_days_ghcnd_'+str(thresh_choose)+'_'+str(area_threshold_choose)+'_'+season_choose+'.nc',mode='w',format='NETCDF4',unlimited_dims='station')